In [1]:
import requests
import pandas as pd
from config import CENSUS_API_KEY

In [42]:
def get_census_data(table_name, state_fips, county_fips):
    api_key = CENSUS_API_KEY
    url = "https://api.census.gov/data/2022/acs/acs5"

    params = {
        "get": f"NAME,group({table_name})",
        "for": f"county:{county_fips}",
        "in": f"state:{state_fips}",
        "key": api_key,
    }

    response = requests.get(url, params=params)
    print(f"{url}?get=NAME,group({table_name})&for=county:{county_fips}&in=state:{state_fips}&key={api_key}")
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        return df
    else:
        print("Failed to retrieve data:", response.status_code)


In [48]:
def summarize_census_data(df):
    # Print the full DataFrame column names and first row to understand what each column contains
    print(df.columns)
    print(df.iloc[0])

    # If needed, print detailed descriptions for specific columns
    # Here you might display descriptions if you had them loaded from an external source or documentation
    
    # Example of summarizing data for one of the variables
    print("\nSummary for Total Commuting Population (B08134_001E):")
    print(df['B08134_001E'].describe())

    # Example of displaying unique values for an annotation column if non-null values exist
    if df['B08134_001EA'].notna().any():
        print("\nAnnotations for Total Commuting Population:")
        print(df['B08134_001EA'].unique())

In [47]:
variables = [
    'B08134',  # Total
    # 'B08134_003E',  # Car, truck, or van - Less than 5 minutes
    # 'B08134_011E',  # Car, truck, or van - 20 to 24 minutes
    # 'B08134_021E',  # Public transportation (excluding taxicab) - Less than 5 minutes
    # 'B08134_029E',  # Public transportation (excluding taxicab) - 20 to 24 minutes
    # 'B08134_045E',  # Walked - Time not specified
]
tables = ','.join(variables)
df = get_census_data(tables, "47", "065")

https://api.census.gov/data/2022/acs/acs5?get=NAME,group(B08134)&for=county:065&in=state:47&key=d5bd87e9fe4db19d2c5c5150ec106748a006a2dd


In [59]:
columns_to_be_renamed = {'B08134_001E':'total_estimate', 'B08134_001M': 'total_margin', 
                         'B08134_002E':'less_than_10_mins_estimate', 'B08134_002M':'less_than_10_mins_margin', 
                         'B08134_003E':'10_to_14_mins_estimate', 'B08134_003M':'10_to_14_mins_margin',
                         'B08134_004E':'15_to_19_mins_estimate', 'B08134_004M':'15_to_19_mins_margin',
                         'B08134_005E':'20_to_24_mins_estimate', 'B08134_005M':'20_to_24_mins_margin',
                         'B08134_006E':'25_to_29_mins_estimate', 'B08134_006M':'25_to_29_mins_margin',
                         'B08134_007E':'30_to_34_mins_estimate', 'B08134_007M':'30_to_34_mins_margin',
                         'B08134_008E':'35_to_44_mins_estimate', 'B08134_008M':'35_to_44_mins_margin',
                         'B08134_009E':'44_to_59_mins_estimate', 'B08134_009M':'44_to_59_mins_margin',
                         'B08134_010E':'60_or_more_mins_estimate', 'B08134_010M':'60_or_more_mins_margin',
                         'B08134_011E':'personal_estimate', 'B08134_011M':'personal_margin',
                         'B08134_021E':'drove_alone_estimate', 'B08134_021M':'drove_alone_margin',
                         'B08134_031E':'carpooled_estimate', 'B08134_031M':'carpooled_margin',
                         'B08134_041E':'two_person_carpool_estimate', 'B08134_041M':'two_person_carpool_margin',
                         'B08134_051E':'three_or_more_person_carpool_estimate', 'B08134_051M':'three_or_more_person_carpool_margin',
                         'B08134_061E':'public_transit_estimate', 'B08134_061M':'public_transit_margin',
                         'B08134_071E':'bus_estimate', 'B08134_071M':'bus_margin',
                         'B08134_081E':'subway_estimate', 'B08134_081M':'subway_margin',
                         'B08134_091E':'long_dist_train_estimate', 'B08134_091M':'long_dist_train_margin',
                         'B08134_101E':'walked_estimate', 'B08134_101M':'walked_margin',
                         'B08134_111E':'taxi_motorbike_bicycle_others_stimate', 'B08134_111M':'taxi_motorbike_bicycle_others_margin',
                         } 

In [61]:
df.rename(columns_to_be_renamed, axis=1)

,NAME,total_estimate,B08134_001EA,total_margin,B08134_001MA,less_than_10_mins_estimate,B08134_002EA,less_than_10_mins_margin,B08134_002MA,10_to_14_mins_estimate,...,B08134_119M,B08134_119MA,B08134_120E,B08134_120EA,B08134_120M,B08134_120MA,GEO_ID,NAME,state,county
0,"Hamilton County, Tennessee",159014,None,2482,None,18543,None,1150,None,23120,...,83,None,160,None,85,None,0500000US47065,"Hamilton County, Tennessee",47,065
